In [1]:
%%javascript
require(["codemirror/keymap/sublime", "notebook/js/cell", "base/js/namespace"],
    function(sublime_keymap, cell, IPython) {
        cell.Cell.options_default.cm_config.keyMap = 'sublime';
    
        var cells = IPython.notebook.get_cells();
        for(var cl=0; cl< cells.length ; cl++){
            cells[cl].code_mirror.setOption('keyMap', 'sublime');
        }
    }
);

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# i had to install this beauty https://github.com/matplotlib/ipympl
# to make following to work ...
%matplotlib widget
import ipywidgets as widgets

# %matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import os
import re
import numpy as np
import pandas as pd
import cooler

import itertools

In [4]:
import bioframe
from bioframe.io import resources

# cooltools used dekkerlab-fork, develop-branch, commit-af508bba9c6577b8271b716ba27c330bead15981

from cooler.tools import split, partition
from functools import partial
from cooltools import expected

from cooltools import numutils
from cooltools.eigdecomp import cooler_cis_eig

from helper_module import fillcolor_compartment_style, to_uscs

In [90]:
# READ SOME COMPARTMENTS IN ...
# we have only 100kb externally called compartments ...
binsize = 100_000
res_human = f"{int(binsize/1000)}kb"

comp_path = "/home/venevs/DOTS_TESTING/alv-chicken/MNP/analysis/compartment"
comp_name = f"poolMNP-bettercis.{res_human}.eigs.cis.vecs.txt"


cmp_vals50kb = None

cmp = pd.read_csv(os.path.join(comp_path,comp_name),sep='\t')

chrom="chr1"
# get chromosomal compartments and interpolated version of those ...
cmp_vals = cmp[cmp["chrom"]==chrom]['E1'].values
# # interpolated_cmp = get_proper_interpolated_compartments(clr, cmp_vals, chrom)
# cmp_vals = cmp_vals[::10]

In [91]:
# ls ../newref-al1
# all-allele-spe-1percent-highcis.galGal5.mapq_30.1000.mcool
# all-allele-spe-2percent-highcis.galGal5.mapq_30.1000.mcool
# clone16highcis.galGal5.mapq_30.1000.mcool
# clone17highcis.galGal5.mapq_30.1000.mcool

In [92]:
binsize = 50_000

get_cpath = lambda p,c: os.path.join(p,c+f"::/resolutions/{binsize}")

c1 = cooler.Cooler(get_cpath("../newref-al1/","clone17highcis.galGal5.mapq_30.1000.mcool"))
c2 = cooler.Cooler(get_cpath("../newref-al2/","clone17highcis.galGal5.mapq_30.1000.mcool"))


In [93]:
# make a bunch of decisions/assignments, based on the binsize of the cooler:

if binsize == 50_000:
    cmp_vals_tmp = np.zeros(2*len(cmp_vals))
    cmp_vals_tmp[0::2] = cmp_vals
    cmp_vals_tmp[1::2] = cmp_vals
    cmp_vals = cmp_vals_tmp
    bad_bins =  np.r_[np.arange(180,189),np.arange(650,655),np.arange(1440,1446)]
elif binsize == 100_000:
    bad_bins =  np.r_[np.arange(90,95),np.arange(325,330)]
    cmp_vals = cmp_vals
elif binsize == 250_000:
    bad_bins =  np.array([9*4,9*4+1,32*4,32*4+1])
    if cmp_vals50kb is None:
        cmp_vals = cmp_vals[::3] # probably wrong ...
    else:
        cmp_vals = cmp_vals50kb[::5].copy()
elif binsize == 500_000:
    bad_bins =  np.array([9*2,9*2+1,32*2,32*2+1,72*2,72*2+1])
    cmp_vals = cmp_vals[::5]
elif binsize == 1_000_000:
    bad_bins =  np.array([9,32,72])
    cmp_vals = cmp_vals[::10]
else:
    print("uh oh - dunno !")
#  
# insertion site, bin-id:    
#

if binsize == 50_000:
    ins_row = 1451 # obvious only in raw data, balaned has al1 and al2 separated !
elif binsize == 100_000:
    ins_row = 725 # 290*2.5
elif binsize == 250_000:
    ins_row = 290 # ok
elif binsize == 500_000:
    ins_row = 145
elif binsize == 1_000_000:
    ins_row = 72
else:
    print("uh oh - dunno where is the insertion!")

In [94]:
parm = ("chr1",0,74626886)
qarm = ("chr1",75126886,196202544)
centro = ("chr1",74626886,75126886)
balance = True

# bad_bins = None
ignore_diags = 2
theregion=parm
theregion_name="parm"

bad_bin_region = ("chr1",9000000,9500000)

weight_name = "weight"
weight1 = weight_name + "1"
weight2 = weight_name + "2"
transforms = {"balanced": lambda p: p["count"] * p[weight1] * p[weight2]}

In [95]:
print(c1.info["nbins"],c2.info["nbins"])
print(c1.info["nnz"],c2.info["nnz"])
print(c1.info["sum"],c2.info["sum"])

20431 20431
83083621 83083180
498961285 498965657


In [96]:
# insert existing eigen track as a phasing track:
bins = c1.bins()[:].copy()
# no support for bad bins in eigdecomp ...
cmp_phase = np.zeros(len(bins))
cmp_phase[:len(cmp_vals)] = cmp_vals
bins.insert(4,"cmp",cmp_phase)

eigvals1, eigvec_table1 = cooler_cis_eig(
    clr=c1,
    bins=bins,
    regions= [parm, qarm],
    n_eigs=2,
    bad_bins = bad_bins,
    phasing_track_col="cmp",
    clip_percentile=99.9,
    sort_metric=None,
)



# insert existing eigen track as a phasing track:
bins = c2.bins()[:].copy()
# no support for bad bins in eigdecomp ...
cmp_phase = np.zeros(len(bins))
cmp_phase[:len(cmp_vals)] = cmp_vals
bins.insert(4,"cmp",cmp_phase)

eigvals2, eigvec_table2 = cooler_cis_eig(
    clr=c2,
    bins=bins,
    regions= [parm, qarm],
    n_eigs=2,
    bad_bins = bad_bins,
    phasing_track_col="cmp",
    clip_percentile=99.9,
    sort_metric=None,
)



In [124]:
# plot it, just in case:
f = plt.figure(figsize=(10,5))
ax1 = f.add_subplot(3,1,1)
ax2 = f.add_subplot(3,1,2)
ax3 = f.add_subplot(3,1,3)
print(len(eigvec_table1))



pq_range_zoom=widgets.IntRangeSlider(
        value=c1.extent(chrom),
        min=0,
        max=c1.extent(chrom)[1],
        step=1,
        continuous_update=False)
@widgets.interact(pq_range_zoom=pq_range_zoom)
def update(pq_range_zoom):
    ax1.clear()
    ax2.clear()
    ax3.clear()
    
    ax1.set_xlim(*pq_range_zoom)
    ax2.set_xlim(*pq_range_zoom)
    ax3.set_xlim(*pq_range_zoom)

    fillcolor_compartment_style(eigvec_table1["E1"].values, ax1)
    #     # fillcolor_compartment_style(cmp_vals, ax)
    #     for bb in bad_bins:
    #         ax.axvline(bb)
    fillcolor_compartment_style(eigvec_table2["E1"].values, ax2)
#     fillcolor_compartment_style(np.log(np.abs((eigvec_table1["E1"]-eigvec_table2["E1"]).values)), ax3)
    fillcolor_compartment_style((eigvec_table1["E1"]-eigvec_table2["E1"]).values, ax3)
    



/home/venevs/miniconda3/envs/newexp/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

20431


interactive(children=(IntRangeSlider(value=(0, 3925), continuous_update=False, description='pq_range_zoom', ma…

### seeming indifference between EV1's of the same dataset mapped to 2 different genomes

`EV1(al1)-EV1(al2)` appear `=0` upstream of the insertion site, but i think now it's just a relative thing - it isn't `0` exactly - visible on `log`-scale ...

the difference is indeed very small, but that's I guess fine and expected given the fact only small portion of the heatmap if affected by the change ...

see the diagram of affected areas below

In [120]:
# plot it, just in case:
f1 = plt.figure(figsize=(3,3))
ggg, = f1.add_gridspec(1,1)
ax = f1.add_subplot(ggg)

mat = np.zeros((100,100))

# highlight diagonal for visual guidance:
mat[np.diag_indices(mat.shape[0])] +=1

# highlight "affected" areas:
mat[:,-10:] += 1
mat[-10:,:] += 1
ax.imshow(mat)

/home/venevs/miniconda3/envs/newexp/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# # redefine cmp_vals:
# cmp_vals = eigvec_table1[eigvec_table1["chrom"]==chrom]["E1"].values
# if binsize == 50_000:
#    cmp_vals50kb = cmp_vals.copy()